In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from PIL import Image
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader

In [2]:
epochs = 10
batch_size = 512

device = ("cuda" if torch.cuda.is_available() else "cpu")
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] # 총 10개의 클래스

In [3]:
device

'cuda'

In [4]:
transform = transforms.Compose(transforms=[transforms.Resize(size=(227, 227)),
                                           transforms.ToTensor()])

training_data = datasets.FashionMNIST(root="data", train=True, transform=transform, download=True)
validation_data = datasets.FashionMNIST(root="data", train=False, transform=transform, download=True)

In [5]:
training_loader = DataLoader(dataset=training_data, batch_size=64, shuffle=True)
validation_loader = DataLoader(dataset=validation_data, batch_size=64, shuffle=True)

In [6]:
def matplotlib_imshow(image):
    image = image.mean(dim=0)
    image = image / 2 + 0.5
    np_image = image.numpy()
    plt.imshow(X=np_image, cmap="Greys")

In [7]:
dataiter = iter(training_loader)
images, labels = next(dataiter)

In [9]:
class AlexNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
        
            nn.Conv2d(in_channels=1, out_channels=96, kernel_size=11, stride=4, padding=0), 
            # 4D tensor : [number_of_kernels, input_channels, kernel_width, kernel_height] 
            # = 96x1x11x11
            # input size : 1x227x227
            # input size 정의 : (N, C, H, W) or (C, H, W)
            # W' = (W-F+2P)/S + 1
            # 55x55x96 feature map 생성 (55는 (227-11+1)/4)
            # 최종적으로 227 -> 55
            nn.ReLU(), # 96x55x55
            nn.MaxPool2d(kernel_size=3, stride=2) 
            # 55 -> (55-3+1)/2 = 26.5 = 27
            # 96x27x27 feature map 생성

        ) 
        self.conv2 = nn.Sequential(
            nn.Conv2d(96, 256, 5, 1, 2), # in_channels: 96, out_channels: 256, kernel_size=5x5, stride=1, padding=2
            # kernel 수 = 48x5x5 (드롭아웃을 사용했기 때문에 96/2=48) 형태의 256개
            # 256x27x27
            nn.ReLU(),
            nn.MaxPool2d(3, 2) # 27 -> 13
            # 256x13x13
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU() # 13 유지
            # 384x13x13
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.ReLU() # 13 유지
            # 384x13x13
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2) # 13 -> 6
            # 256x6x6
        )
        
        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 10)

    def forward(self, x): # input size = 3x227x227
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out) # 64x4096x1x1
        out = out.view(out.size(0), -1) # 64x4096
        
        out = F.relu(self.fc1(out))
        out = F.dropout(out, 0.5)
        out = F.relu(self.fc2(out))
        out = F.dropout(out, 0.5)
        out = self.fc3(out)
        out = F.log_softmax(out, dim=1)
        
        return out

In [16]:
model = AlexNet().to(device=device)
criterion = F.nll_loss
optimizer = optim.Adam(params=model.parameters())

In [17]:
from torchsummary import summary

summary(model=model, input_size=(1, 227, 227), batch_size=batch_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [512, 96, 55, 55]          11,712
              ReLU-2          [512, 96, 55, 55]               0
         MaxPool2d-3          [512, 96, 27, 27]               0
            Conv2d-4         [512, 256, 27, 27]         614,656
              ReLU-5         [512, 256, 27, 27]               0
         MaxPool2d-6         [512, 256, 13, 13]               0
            Conv2d-7         [512, 384, 13, 13]         885,120
              ReLU-8         [512, 384, 13, 13]               0
            Conv2d-9         [512, 384, 13, 13]       1,327,488
             ReLU-10         [512, 384, 13, 13]               0
           Conv2d-11         [512, 256, 13, 13]         884,992
             ReLU-12         [512, 256, 13, 13]               0
        MaxPool2d-13           [512, 256, 6, 6]               0
           Linear-14                [51

In [18]:
def train(model, device, train_loader, optimizer, epoch):
    
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        # enumberate() : 인덱스와 원소로 이루어진 튜플(tuple)을 만들어줌
        target = target.type(torch.LongTensor)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # 항상 backpropagation 하기전에 미분(gradient)을 zero로 만들어주고 시작해야 한다.
        output = model(data)
        loss = criterion(output, target) # criterion = loss_fn
        loss.backward() # Computes the gradient of current tensor w.r.t. graph leaves
        optimizer.step() # step() : 파라미터를 업데이트함
        if (batch_idx + 1) % 30 == 0:
            print("Train Epoch:{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [19]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)  # -> mean
        print("\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
        print('='*50)

In [20]:
for epoch in range(1, 2):
    train(model, device, training_loader, optimizer, epoch)
    test(model, device, validation_loader)

Train Epoch:1 [1856/60000 (3%)]	Loss: 1.598908
Train Epoch:1 [3776/60000 (6%)]	Loss: 1.149229
Train Epoch:1 [5696/60000 (9%)]	Loss: 0.965210
Train Epoch:1 [7616/60000 (13%)]	Loss: 0.910678
Train Epoch:1 [9536/60000 (16%)]	Loss: 0.879698
Train Epoch:1 [11456/60000 (19%)]	Loss: 0.708798
Train Epoch:1 [13376/60000 (22%)]	Loss: 0.476133
Train Epoch:1 [15296/60000 (25%)]	Loss: 0.530906
Train Epoch:1 [17216/60000 (29%)]	Loss: 0.558219
Train Epoch:1 [19136/60000 (32%)]	Loss: 0.667198
Train Epoch:1 [21056/60000 (35%)]	Loss: 0.900905
Train Epoch:1 [22976/60000 (38%)]	Loss: 0.441009
Train Epoch:1 [24896/60000 (41%)]	Loss: 0.517382
Train Epoch:1 [26816/60000 (45%)]	Loss: 0.520099
Train Epoch:1 [28736/60000 (48%)]	Loss: 0.403478
Train Epoch:1 [30656/60000 (51%)]	Loss: 0.470538
Train Epoch:1 [32576/60000 (54%)]	Loss: 0.394310
Train Epoch:1 [34496/60000 (57%)]	Loss: 0.407234
Train Epoch:1 [36416/60000 (61%)]	Loss: 0.329897
Train Epoch:1 [38336/60000 (64%)]	Loss: 0.368430
Train Epoch:1 [40256/60000 (